In [ ]:
import requests
import random
import time
import os
import urllib.parse
import pandas as pd
from datetime import datetime
pd.set_option('display.max_rows', 1000)

In [ ]:
s = requests.Session()
s.get('https://profile.callofduty.com/cod/login')
data = {'username': os.environ.get('COD_EMAIL'), 
        'password': os.environ.get('COD_PW'), 
        'remember_me': 'true', 
        '_csrf': s.cookies['XSRF-TOKEN']}
s.post('https://profile.callofduty.com/do_login?new_SiteId=cod', params=data)

The matches endpoint is made up of different parts  
https://my.callofduty.com/api/papi-client/crm/cod/:version/title/:game/platform/:platform/gamer/:username/matches/:mode/start/:start/end/:end/details

### Version
API Version - Mostly v2 should work for Warzone.

### game
Game - "mw" for Modern Warfare, "wwii" for WWII, "bo4" for Black Ops 4

### platform
Platform associated with username - "uno" for Activision, "battle" for Battle.net, psn for Playstation, xbl for XBOX Live "steam" for WWII

### username
URI encoded string of the platform-specific username (eg: Viioozz for my psn)

### mode
Game Mode - "wz" for Warzone or "mp" for Multiplayer

### start
UNIX Timestamp

Can remain at 0, or unless you want only the games between certain timeframes, can be interesting for killrace like tournamements where only the games during a certain time window count. Defaults to 0.

### end
UNIX Timestamp in milliseconds, defaults to 0


Let's call the api without providing timestamps:

In [ ]:
latest_matches_response = s.get('https://www.callofduty.com/api/papi-client/crm/cod/v2/title/mw/platform/xbl/gamer/coltie119/matches/wz/start/0/end/0/')
latest_matches_response.json()

You'll see that you get a summary of the latest 20 games by default. If you want the details of every game you have to add '/details' to the endpoint.

In [ ]:
latest_matches_response = s.get('https://www.callofduty.com/api/papi-client/crm/cod/v2/title/mw/platform/psn/gamer/Viioozz/matches/wz/start/0/end/0/details')
latest_matches_response.json()

## URL Encode

If you want to use other platforms like battle.net, keep in mind you'll have to url encode the username if they contain a hash symbol for example or other 'special' characters.

In [ ]:
username = 'Viöo$'
urllib.parse.quote(nickname)

In [ ]:
username = 'Vioo#21794'
urllib.parse.quote(nickname)

In [ ]:
s.get('https://www.callofduty.com/api/papi-client/crm/cod/v2/title/mw/platform/battle/gamer/{username}/matches/wz/start/0/end/0')

## UNIX Timestamps

Unix time is a system for describing a point in time. It is the number of seconds that have elapsed since the Unix epoch, minus leap seconds; the Unix epoch is 00:00:00 UTC on 1 January 1970. It's a way of encoding time in a numeric way.

Let's get the current time and use it to call the API.

! Keep in mind, for the api we need MILLISECONDS, so we multiply by 1000.

In [ ]:
current_ts = int(time.time() * 1000)
current_ts

In [ ]:
resp_matches = s.get(f'https://www.callofduty.com/api/papi-client/crm/cod/v2/title/mw/platform/psn/gamer/Viioozz/matches/wz/start/0/end/{current_ts}/details?limit=20')

In [ ]:
matches = resp_matches.json()['data']['matches']

In [ ]:
len(matches)

In [ ]:
matches

## Entire Match History

How to pull all your matches as we only have 20 in one response. The way I solved this is to write a while loop that goes through my match history and stops when less than 20 matches are in a response meaning we are pulling in the last batch.  
For every subsequent call I use the timestamp of the latest game in the response as the 'end' parameter, so then I get all 20 matches before that last game.

In [ ]:
tsLastGame = '0'
matchesFound = True
matches = []
while matchesFound:
    resp_matches = s.get('https://www.callofduty.com/api/papi-client/crm/cod/v2/title/mw/platform/battle/gamer/Vioo%2321794/matches/wz/start/0/end/' + str(tsLastGame) + '000/details?limit=20')
    if len(resp_matches.json()['data']['matches']) == 20:
        new_matches = resp_matches.json()['data']['matches']
        tsLastGame = new_matches[-1]['utcStartSeconds']
        matches += new_matches
    else:
        matchesFound = False
    time.sleep(0.2)
        
len(matches)

## Match Modes

## Calculate best games using Pandas DataFrames

In [ ]:
df_matches = pd.DataFrame(matches)
df_matches = df_matches[~df_matches['mode'].str.contains('dmz')]
len(df_matches)
df_matches = pd.concat([df_matches.drop(['playerStats'], axis=1), df_matches['playerStats'].apply(pd.Series)], axis=1)
df_matches.sort_values(by=['kills', 'damageDone'], ascending=False, inplace=True)
csv_top = df_matches[['kills', 'deaths', 'gulagKills', 'gulagDeaths', 'damageDone', 'damageTaken', 'mode', 'teamPlacement']]
csv_top.to_csv("./top.csv")

In [ ]:
print(df_matches['kills'].sum())
print(df_matches['deaths'].sum())
print(df_matches['gulagDeaths'].sum())
#print(df_matches[df_matches['gulagDeaths'] > 1].count())
df_matches['kills'].sum()/df_matches['deaths'].sum()

In [ ]:
df_matches['gulagKills'].sum()

In [ ]:
df_matches[df_matches['gulagKills'] >= 1]['kills']

In [ ]:
df_matches.groupby(['mode']).count().sort_values('mode')

In [ ]:
print(matches[-1])

In [ ]:
matchIds = []
for match in matches:
    matchIds.append(match['rankedTeams'])
print(len(set(matchIds)))

In [ ]:
matchModes = []
for match in matches:
    matchModes.append(match['gameType'])
print(len(matchModes))
modes = set(matchModes)
print(modes)

In [ ]:
for match in matches:
    if match['mode'] == "brtdm_113":
        print(match['playerStats']['gulagDeaths'])

In [ ]:
totalWZMatches = 0
totalGulags = 0
totalGulagKills = 0
totalGulagDeaths = 0
totalKills = 0
totalDeaths = 0
for match in matches:
    gulagKills = match['playerStats'].get('gulagKills', 0.0)
    gulagDeaths = match['playerStats'].get('gulagDeaths', 0.0)
    print(gulagDeaths)
    if match['mode'] not in ["br_dmz_plnbld", "brtdm_113", "br_dmz_104", "br_dmz_38", "br_dmz_85"]:
        print(str(match['playerStats']['kills']) + "/" + str(match['playerStats']['deaths']))
        if gulagKills != 0.0 or gulagDeaths != 0.0:
            totalGulags += 1
            if gulagKills >= 1.0:
                totalGulagKills += 1
                print("Gulag Won!")
            if gulagDeaths >= 1.0:
                totalGulagDeaths += 1
                print("Gulag Lost!")
        totalWZMatches += 1
        totalKills += match['playerStats']['kills']
        totalDeaths += match['playerStats']['deaths']
print(totalWZMatches)
print(totalGulags)
print(totalGulagKills)
print(totalGulagDeaths)
print(totalKills)
print(totalDeaths)
    

In [ ]:
727/800

In [ ]:
date_start = datetime(2000,1,1,0,0)
date_start = int(time.mktime(date_start.timetuple()))
date_start